In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders.mongodb import MongodbLoader
import nest_asyncio

nest_asyncio.apply()
load_dotenv()

In [ ]:
loader = MongodbLoader(
    connection_string=os.environ['MONGODB_URI'],
    db_name=os.environ['MONGODB_DB'],
    collection_name=os.environ['MONGODB_COLL'],
    filter_criteria={},
    field_names=["title", "plot"]
)
docs = loader.load()

print(len(docs))
docs[0]

In [ ]:
from pymongo import MongoClient
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
client = MongoClient(os.environ['MONGODB_URI'], appname="devrel.content.langchain_llamaIndex.python")
collection = client.get_database(os.environ['MONGODB_DB']).get_collection(os.environ['MONGODB_VECTOR_COLL_LANGCHAIN'])

vector_search = MongoDBAtlasVectorSearch.from_documents(
  documents=docs,
  embedding=OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY']),
  collection=collection,
  index_name=os.environ['MONGODB_VECTOR_INDEX'])

In [ ]:
# perform a similarity search on the ingested documents
prompt='What is the best horror movie to watch?'
docs_with_score = vector_search.similarity_search_with_score(query=prompt,k=1)

llm = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'])

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a movie recommendation engine which posts a concise and short summary on relevant movies."),
    ("user", "List of movies: {input}")
])

# Create an LLMChain
chain = LLMChain(
    llm=llm,
    prompt=prompt_template
)

# Prepare the input for the chat model
input_docs = "\n".join([doc.page_content for doc, _ in docs_with_score])

# Invoke the chain with the input documents
response = chain.invoke({"input": input_docs})
print(response['text'])